# Produce Driver Standings

In [0]:
%run "../includes/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results.parquet")

In [0]:
race_results_df.show(5)

+---------+--------------------+---------+----------------+----------------+-------------+------------------+----------+----+-----------+-----------+------+--------+
|race_year|           race_name|race_date|circuit_location|     driver_name|driver_number|driver_nationality|      team|grid|fastest_lap|  race_time|points|position|
+---------+--------------------+---------+----------------+----------------+-------------+------------------+----------+----+-----------+-----------+------+--------+
|     1990|Hungarian Grand Prix|     null|        Budapest| Thierry Boutsen|         null|           Belgian|  Williams|   1|       null|1:49:30.597|   9.0|       1|
|     1990|Hungarian Grand Prix|     null|        Budapest|    Ayrton Senna|         null|         Brazilian|   McLaren|   4|       null|     +0.288|   6.0|       2|
|     1990|Hungarian Grand Prix|     null|        Budapest|   Nelson Piquet|         null|         Brazilian|  Benetton|   9|       null|    +27.893|   4.0|       3|
|   

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as f

In [0]:
driver_standings_df = race_results_df.groupBy("race_year", "driver_name", "driver_nationality", "team") \
                        .agg(f.sum("points").alias("total_points"), f.count(f.when(f.col("position") == 1, True)).alias("wins"))

In [0]:
display(driver_standings_df.filter("race_year = 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins
2020,Lance Stroll,Canadian,Racing Point,75.0,0
2020,Kevin Magnussen,Danish,Haas F1 Team,1.0,0
2020,Antonio Giovinazzi,Italian,Alfa Romeo,4.0,0
2020,Carlos Sainz,Spanish,McLaren,105.0,0
2020,Lewis Hamilton,British,Mercedes,347.0,11
2020,Nicholas Latifi,Canadian,Williams,0.0,0
2020,Pierre Gasly,French,AlphaTauri,75.0,1
2020,Jack Aitken,British,Williams,0.0,0
2020,Romain Grosjean,French,Haas F1 Team,2.0,0
2020,Daniil Kvyat,Russian,AlphaTauri,32.0,0


In [0]:
driver_rank_spec = Window.partitionBy("race_year").orderBy(f.desc("total_points"), f.desc("wins"))

In [0]:
final_df = driver_standings_df.withColumn("rank", f.rank().over(driver_rank_spec))

In [0]:
display(final_df.filter("race_year = 2020"))

race_year,driver_name,driver_nationality,team,total_points,wins,rank
2020,Lewis Hamilton,British,Mercedes,347.0,11,1
2020,Valtteri Bottas,Finnish,Mercedes,223.0,2,2
2020,Max Verstappen,Dutch,Red Bull,214.0,2,3
2020,Sergio Pérez,Mexican,Racing Point,125.0,1,4
2020,Daniel Ricciardo,Australian,Renault,119.0,0,5
2020,Carlos Sainz,Spanish,McLaren,105.0,0,6
2020,Alexander Albon,Thai,Red Bull,105.0,0,6
2020,Charles Leclerc,Monegasque,Ferrari,98.0,0,8
2020,Lando Norris,British,McLaren,97.0,0,9
2020,Pierre Gasly,French,AlphaTauri,75.0,1,10


In [0]:
final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings.parquet")